In [4]:
import re
colors = {
    "00": "",
    "01": "\\MULTICOLORONE",
    "10": "\\SPRITECOLOR",
    "11": "\\MULTICOLORTWO",
}
sprites_files = ["../iridisalpha/src/graphics/sprites.asm",
                 "../iridisalpha/src/graphics/enemy_sprites.asm",
                "../iridisalpha/src/graphics/bonusphase_sprites.asm",]
sprites = {}
for sprites_file in sprites_files:
    input_file = open(sprites_file,'r')
    sprite_data = []
    for l in input_file.readlines():
        if "SPRITE" in l:
            if sprite_data:
                sprites[sprite_name] = sprite_data
            sprite_name = l[22:].strip()
            sprite_data = []
            continue

        m = re.findall(r"[0-1]{24}",l)
        if not m:
            continue
        bits = m[0]
        for i in range(0,23,2):
            bitpair = bits[i:i+2]
            sprite_data += [colors[bitpair]]
            sprite_data += [colors[bitpair]]
sprites[sprite_name] = sprite_data


In [2]:

header = """
\\begin{figure}[H]
  {
    \\setlength{\\tabcolsep}{3.0pt}
    \\setlength\cmidrulewidth{\heavyrulewidth} % Make cmidrule = 
    \\begin{adjustbox}{width=3cm,center}
      \\begin{tikzpicture}
"""

footer = """
      \\end{tikzpicture}
    \\end{adjustbox}
  }\\caption{%SPRITE_NAME%}
\\end{figure}
"""

In [5]:

def createLatexSpriteGrid(sprite_name):
    gl = 24
    gh = 21
    glc = gl - 1
    ghc = gh - 1
    
    sp = sprites[sprite_name]
    sl = len(sp)
    output = header + '\n'
    grid = f"\\draw[step=1.0,gray,thin] (0,0) grid ({gl},{gh});"
    output += f"\t{grid}\n"
    for i,v in enumerate(sp):
        x = (i % gl) if i >= glc else i
        y = (ghc - int(i/gl)) if i >= ghc else ghc
        if not v:
            continue
        output += f'\t\\fill[{v}] ({x},{y}) rectangle ++ (1,1);\n'
    output += footer.replace("%SPRITE_NAME%", sprite_name.replace("_","\_"))
    return output

#names = ["FLYING_SAUCER1","FLYING_SAUCER2","FLYING_SAUCER3"]
for nm in sprites:
    output = createLatexSpriteGrid(nm)
    output_file = open(f"../src/sprites/{nm}.tex",'w')
    output_file.write(output)
    output_file.close()